# Query the Barb API using python

In this demo we will show you how to query the Barb API using Python. We will do this first without making use of the pybarb package (as you may wish to develop your own software from scratch). We will then show you how to use the pybarb package to make the same query.

Note the full API documentation can be found [here](https://barb-api.co.uk/api-docs). 

It might also be useful to consult the [Getting Started](https://barb-api.co.uk/api-docs#section/Getting-started) section for information about authentication and basic API usage.

## Querying the API without pybarb

First we use the code and instructions from the notebook "Connecting to the BARB API using Python" to connect to the API.

In [1]:
import requests
import json

# Set the working directory
working_directory = '/path/to/your/dir/'

# Get the access token
with open(working_directory + "creds.json") as file:
    creds = json.load(file)
api_root = "https://barb-api.co.uk/api/v1/"
token_request_url = api_root + "auth/token/"
response = requests.post(token_request_url, data = creds)
access_token = json.loads(response.text)['access']

# Set the headers
headers = {'Authorization': 'Bearer {}'.format(access_token)}

We now have everything we need to query the BARB API. Let's start by getting the programme ratings for all programmes broadcast on BBC One to the BBC Network panel between 25th and 31st December 2022. 

We use [the endpoint documentation](https://barb-api.co.uk/api-docs#tag/Events/operation/ProgrammeRatings) to understand which query parameters can be used and what values they can take. We set the parameters for the query as follows:

In [2]:
params = {"min_transmission_date": "2022-12-25","max_transmission_date":"2022-12-31", 
          "station_code": 10, 
          "panel_code": 50,
         "consolidated": True, "limit": 5000}

Now we can query the endpoint by passing it the parameters and the headers we just created. We will use the requests library to do this. We will also use the json library to format the response as a JSON object.

In [3]:
api_url = api_root + "programme_ratings/"
r = requests.get(url=api_url, params=params, headers=headers)
api_data = r.json()

The data has a nested structure which is explained in the API documentation. 

This data can be flattened using the json_normalize() function from the pandas library. This function takes a JSON object and flattens it into a pandas DataFrame. 

In [5]:
from pandas import json_normalize
json_normalize(api_data['events']).head()


,transmission_log_programme_name,programme_type,programme_duration,spans_normal_day,broadcaster_transmission_code,live_status,uk_premier,broadcaster_premier,repeat,platforms,...,programme_content.content_name,programme_content.barb_content_id,programme_content.broadcaster_content_id,programme_content.metabroadcast_information.metabroadcast_content_id,programme_content.episode.episode_number,programme_content.episode.episode_name,programme_content.series.series_number,programme_content.series.number_of_episodes,programme_content.series.broadcaster_series_id,programme_content.genre
0,MICHAEL MCINTYRE'S CHRISTMAS WHEEL,programme,57,False,15056800065812,unknown,True,True,False,"[Analogue Terrestrial, Digital Terrestrial, An...",...,"Michael McIntyre's The Wheel: Series 3, Episode 7",7398800,m001gnjd,p9jg8r,7,Christmas Special,3,12,None,Entertainment
1,SUPERWORM,programme,25,False,31485693446812,unknown,False,False,True,"[Analogue Terrestrial, Digital Terrestrial, An...",...,FILM: Superworm (2021),6140411,m0012w40,pjtpcs,0,Superworm,(2021),0,None,Films
2,EastEnders,programme,59,False,11875724298812,unknown,True,True,False,"[Analogue Terrestrial, Digital Terrestrial, An...",...,EastEnders: Series 2022,7398806,m001gnjl,p9jg8v,0,25/12/2022,(2022),0,None,Drama
3,MRS BROWN'S BOYS CHRISTMAS SPECIAL,programme,29,False,14320869134812,unknown,True,True,False,"[Analogue Terrestrial, Digital Terrestrial, An...",...,"Mrs Brown's Boys: Series 18, Episode 1",7398804,m001gnjn,p9jg8w,1,Shining Mammy,18,2,None,Entertainment
4,GHOSTS CHRISTMAS SPECIAL,programme,30,False,19147778551812,unknown,True,True,False,"[Analogue Terrestrial, Digital Terrestrial, An...",...,Ghosts: Series 2022,7398770,m001gnjg,p9jg8s,0,It's Behind You,(2022),0,None,Entertainment


If the dataset is large then it will be returned over multiple pages. To return all the data we need to loop through the pages and append the data to a list. The following code will return all the data for the last half of 2022.

In [6]:
api_url = api_root + "programme_ratings/"

# The query parameters
params = {"min_transmission_date": "2022-07-01","max_transmission_date":"2022-12-31", 
          "station_code": 10, 
          "panel_code": 50,
         "consolidated": True, "limit": 5000}

# Query the API and turn the response into json
api_data = []
r = requests.get(url=api_url, params=params, headers=headers)
api_data.append(r.json())

while r.headers.__contains__("X-Next"):
    print(f'Downloading the next page from {r.headers["X-Next"]}')
    x_next_url = r.headers["X-Next"]
    r = requests.get(url=x_next_url, headers=headers)
    api_data.append(r.json())


However to properly assemble a table of audience data we need to join the `event` data with the `audience_categories` data. This is straightforward to do using the `merge()` function from the pandas library, however the pybarb package handles all of this for you.

## Querying the API with pybarb

First we connect to the API using the `pybarb` package as described in "Connecting to the Barb API using Python". 

In [7]:
import pybarb as pb

# Create a BarbAPI object and connect
barb_api = pb.BarbAPI(creds)
barb_api.connect()

Now we can use `programme_ratings` method to query the programme ratings endpoint. This method will handle the pagination for us.

In [8]:
bbc_1_progs = barb_api.programme_ratings(min_transmission_date = "2022-12-25", 
                           max_transmission_date = "2022-12-31", 
                           station="BBC1", panel="bbc network", 
                           consolidated=True)

The downloaded data can be viewed in the `api_response_data` attribute.

However it is much more convenient to use the `to_dataframe` method to convert the response to a pandas dataframe. This method will also do things like convert the dates to datetime objects and merge the `event` data with the `audience_categories` data.


In [10]:
bbc_1_progs.to_dataframe()

,panel_region,station_name,programme_name,programme_type,programme_start_datetime,programme_duration_minutes,spans_normal_day,uk_premiere,broadcaster_premiere,programme_repeat,episode_number,episode_name,genre,audience_size_hundreds,date_of_transmission,audience_name,audience_target_size_hundreds
0,BBC Network,BBC1,"Michael McIntyre's The Wheel: Series 3, Episode 7",programme,2022-12-25 18:25:43,57,False,True,True,False,7,Christmas Special,Entertainment,28849,2022-12-25,All Homes,272070
1,BBC Network,BBC1,"Michael McIntyre's The Wheel: Series 3, Episode 7",programme,2022-12-25 18:25:43,57,False,True,True,False,7,Christmas Special,Entertainment,55235,2022-12-25,All Adults,514750
2,BBC Network,BBC1,"Michael McIntyre's The Wheel: Series 3, Episode 7",programme,2022-12-25 18:25:43,57,False,True,True,False,7,Christmas Special,Entertainment,21919,2022-12-25,All Men,250780
3,BBC Network,BBC1,"Michael McIntyre's The Wheel: Series 3, Episode 7",programme,2022-12-25 18:25:43,57,False,True,True,False,7,Christmas Special,Entertainment,31321,2022-12-25,All Houseperson,272070
4,BBC Network,BBC1,"Michael McIntyre's The Wheel: Series 3, Episode 7",programme,2022-12-25 18:25:43,57,False,True,True,False,7,Christmas Special,Entertainment,5348,2022-12-25,All Children aged 4-15,95890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24835,BBC Network,BBC1,Weather for the Week Ahead: Series 2022,programme,2023-01-01 03:14:33,3,False,False,False,True,0,01/01/2023,Weather,0,2023-01-01,"Adults, Lightest Third",171580
24836,BBC Network,BBC1,Weather for the Week Ahead: Series 2022,programme,2023-01-01 03:14:33,3,False,False,False,True,0,01/01/2023,Weather,0,2023-01-01,"Adults, Lightest Sixth",85800
24837,BBC Network,BBC1,Weather for the Week Ahead: Series 2022,programme,2023-01-01 03:14:33,3,False,False,False,True,0,01/01/2023,Weather,0,2023-01-01,"ABC1 Adults, Lightest Third",97120
24838,BBC Network,BBC1,Weather for the Week Ahead: Series 2022,programme,2023-01-01 03:14:33,3,False,False,False,True,0,01/01/2023,Weather,0,2023-01-01,"Adults 16-34, Lightest Third",47380
